# Preprocessing

In [15]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion DNSMOS

In [22]:
import torch
from torchmetrics.audio import DeepNoiseSuppressionMeanOpinionScore

class DNSMOS:
    """
    DNSMOS (Deep Noise Suppression Mean Opinion Score) Modell zur Bewertung der Sprachqualität ohne Referenzsignal.
    
    Quelle:
        - Microsoft Research: DNSMOS – "Deep Noise Suppression Mean Opinion Score"
        - Reddy et al., "DNSMOS: A Non-Intrusive Perceptual Objective Speech Quality Metric to Evaluate Noise Suppressors", 
          in *arXiv preprint arXiv:2010.15258*, 2020.
        - https://github.com/microsoft/DNS-Challenge
    
    Args:
        sample_rate (int): Sampling-Rate des Audiosignals.
        personalized (bool): Ob das Modell störende Sprecher erkennt und in die Bewertung einbezieht.
        device (str): Rechen-Device (z. B. "cpu" oder "cuda").
    
    Returns:
        dict: DNSMOS-Scores für die Bewertung der Sprachqualität (P808_MOS, SIGMOS, BAKMOS, OVRMOS).
    """
    
    def __init__(self, sample_rate=16000, personalized=False, device="cpu"):
        self.model = DeepNoiseSuppressionMeanOpinionScore(fs=sample_rate, personalized=personalized, device=device)
    
    def calculate_dnsmos(self, audio_signal):
        """
        Berechnet die DNSMOS-Scores für ein Audio-Signal.
        
        Args:
            audio_signal (np.array): Vorverarbeitetes Audiosignal (Mono, 16 kHz, Normalisiert).
        
        Returns:
            dict: DNSMOS-Werte {P808_MOS, SIGMOS, BAKMOS, OVRMOS}.
        """
        # Umwandlung in Tensor für TorchMetrics
        audio_tensor = torch.tensor(audio_signal).float()

        # Berechnung der DNSMOS-Scores
        dnsmos_scores = self.model(audio_tensor)

        return {
            "P808_MOS": dnsmos_scores[0].item(),  # Perceptual MOS (Speech Quality)
            "SIGMOS": dnsmos_scores[1].item(),   # Speech Signal Quality
            "BAKMOS": dnsmos_scores[2].item(),   # Background Noise Quality
            "OVRMOS": dnsmos_scores[3].item()    # Overall Quality
        }

# Test

In [21]:
import os

# Verzeichnis mit den Audiodateien
audio_dir = "../audio_files/"  # <-- Hier den korrekten Pfad anpassen

# Alle Audiodateien im Verzeichnis durchsuchen und alphabetisch sortieren
audio_files = sorted([f for f in os.listdir(audio_dir) if f.endswith(".wav")])

# DNSMOS-Modell initialisieren
dnsmos_model = DNSMOS(sample_rate=16000, personalized=False, device="cpu")

# Durch alle Audiodateien iterieren und DNSMOS berechnen
for file_name in audio_files:
    file_path = os.path.join(audio_dir, file_name)

    # Audio vorverarbeiten (Resampling, Mono, Normalisierung)
    audio_signal = preprocess_audio(file_path)

    # DNSMOS-Scores berechnen
    dnsmos_scores = dnsmos_model.calculate_dnsmos(audio_signal)

    # Ergebnisse ausgeben
    if dnsmos_scores:
        print(f"DNSMOS ({file_name}): P808_MOS={dnsmos_scores['P808_MOS']:.2f}, "
              f"SIGMOS={dnsmos_scores['SIGMOS']:.2f}, "
              f"BAKMOS={dnsmos_scores['BAKMOS']:.2f}, "
              f"OVRMOS={dnsmos_scores['OVRMOS']:.2f}")
    else:
        print(f"Fehler: Keine DNSMOS-Werte für {file_name} berechnet.")

DNSMOS (_noise_pink.wav): P808_MOS=2.18, SIGMOS=3.06, BAKMOS=1.37, OVRMOS=1.51
DNSMOS (_noise_white.wav): P808_MOS=2.11, SIGMOS=1.19, BAKMOS=1.14, OVRMOS=1.06
DNSMOS (_signal_constant.wav): P808_MOS=2.21, SIGMOS=2.30, BAKMOS=3.42, OVRMOS=1.98
DNSMOS (_signal_silence.wav): P808_MOS=2.15, SIGMOS=2.51, BAKMOS=3.47, OVRMOS=1.84
DNSMOS (_signal_sine.wav): P808_MOS=2.11, SIGMOS=2.52, BAKMOS=3.41, OVRMOS=2.19
DNSMOS (_snr_03.wav): P808_MOS=2.39, SIGMOS=2.45, BAKMOS=1.37, OVRMOS=1.44
DNSMOS (_snr_10.wav): P808_MOS=2.61, SIGMOS=3.37, BAKMOS=1.84, OVRMOS=2.00
DNSMOS (_snr_20.wav): P808_MOS=2.89, SIGMOS=3.67, BAKMOS=2.99, OVRMOS=2.81
DNSMOS (example1.wav): P808_MOS=2.32, SIGMOS=2.43, BAKMOS=3.68, OVRMOS=2.17
DNSMOS (example2.wav): P808_MOS=2.47, SIGMOS=2.26, BAKMOS=3.74, OVRMOS=2.05
DNSMOS (example3.wav): P808_MOS=2.90, SIGMOS=2.93, BAKMOS=3.76, OVRMOS=2.59
